In [ ]:
import numpy as np
import time as Time
from numba import jit
import matplotlib.pyplot as plt
import pyvista as pv
import torch

In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [ ]:
maskUp = 2.09
maskBottom = 2.6
maskDeep = 2.808

diameter = 60
maskUp_sim = maskUp/maskBottom*diameter
maskDeep_sim = maskDeep/maskBottom*diameter
print(maskUp_sim)
print(maskDeep_sim)

bottom = 100
height = bottom + int(maskDeep_sim)


# vertical mask
film = np.zeros((100, 100, 160, 3))

bottom = 100
height = 140

density = 10

sphere = np.ones((100, 100, 160), dtype=bool)

radius = 30

center = 50
for i in range(sphere.shape[0]):
    for j in range(sphere.shape[1]):
        if np.abs(i-center)*np.abs(i-center) + np.abs(j-center)*np.abs(j-center) < radius*radius:
            sphere[i, j, bottom:height] = 0

film[sphere, 2] = density
film[:, :, height:, :] = 0
film[:, :, 0:bottom, 0] = density # bottom
film[:, :, 0:bottom, 1] = 0 # bottom
film[:, :, 0:bottom, 2] = 0 # bottom

In [ ]:
maskUp = 2.09
maskBottom = 2.6
maskDeep = 2.808

diameter = 260
maskUp_sim = maskUp/maskBottom*diameter
maskDeep_sim = maskDeep/maskBottom*diameter
print(maskUp_sim)
print(maskDeep_sim)

film = np.zeros((300, 300, 400, 3))

bottom = 100
height = bottom + int(maskDeep_sim)

density = 10

sphere = np.ones((300, 300, 400), dtype=bool)

# diameter = 120

center = 150
for k in range(int(diameter/2 - int(maskUp_sim/2))):
    # print(diameter/2 - k)
    radius = diameter/2 - k
    # print('deep', int(bottom + maskDeep_sim/(diameter/2 - maskUp_sim/2)*k))
    bottom_step = int(bottom + maskDeep_sim/(diameter/2 - maskUp_sim/2)*k)
    for i in range(sphere.shape[0]):
        for j in range(sphere.shape[1]):
            if np.abs(i-center)*np.abs(i-center) + np.abs(j-center)*np.abs(j-center) < radius*radius:
                sphere[i, j, bottom_step:bottom_step+int(maskDeep_sim/(diameter/2 - maskUp_sim/2))] = 0

film[sphere, 2] = density
film[:, :, height:, :] = 0
film[:, :, 0:bottom, 0] = density # bottom
film[:, :, 0:bottom, 1] = 0 # bottom
film[:, :, 0:bottom, 2] = 0 # bottom

In [ ]:
# simple for test method2

film = np.zeros((70, 100, 100, 3))

bottom = 10
# film[:, :, 0:bottom] = 10 # bottom
center =50

height = 30
left_side = 30
right_side = 30

film[:, :left_side, 0:height, 0] = 10
film[:, -right_side:, 0:height, 0] = 10


# film[:, 80:121, 0:31] = 10

film[:, :, 0:bottom, 0] = 10 # bottom
film[:, :, height:, 0] = 0 # bottom

In [ ]:
depo1 = torch.Tensor(film[:, :, :, 1]>0).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(film[:, :, :,0]!=0).to_sparse()
substrute = substrute.indices().numpy().T


depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
# p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
etchfilm = film

In [ ]:
etchfilm = np.load('./sputter_etching_paper_depo_1113.npy')
center = 100

In [ ]:
indice = etchfilm[:, :, :, 0] < 10
etchfilm[indice, 0] = 0

In [ ]:
etchfilm.shape

In [ ]:
import Multi_Species_SF_etching

In [ ]:
logname = 'Multi_species_benchmark_1114_method2'
testEtch = Multi_Species_SF_etching.etching(mirror=True,inputMethod='bunch', pressure_pa=0.001, temperature=300, chamberSize=etchfilm.shape,
                                         depoThick=120, center_with_direction=np.array([[etchfilm.shape[0]/2,center,75]]), 
                                         range3D=np.array([[0, etchfilm.shape[0], 0, etchfilm.shape[1], 0, etchfilm.shape[2]]]), InOrOut=[1], yield_hist=np.array([None]),
                                        reaction_type=False, param = [1.6, -0.7], N = 300000, 
                                        sub_xy=[0,0], film=etchfilm, n=1, cellSize=etchfilm.shape, 
                                        celllength=1e-5, kdtreeN=5, tstep=1e-5,
                                        substrateTop=40,posGeneratorType='top', logname=logname)


# etching = testEtch.depo_position_increase_cosVel_normal(125, int(7e6), 2e-3, 5)


In [ ]:
react_table = np.array([[[0.1, -1, 0, 0], [1.0, 0,  0, 0], [0.0, 0, 0, 0]],
                        [[0.8, -1, 1, 0], [0.0, 0,  0, 0], [0.0, 0, 0, 0]],
                        [[1.0,  0, 0, 0], [1.0, 0, -2, 0], [1.0, 0, 0, 0]]])

In [ ]:
react_table = np.array([[[1, 1, 0, 0], [1.0, 0,  0, 0], [0.0, 0, 0, 0]],
                        [[0.8, -1, 1, 0], [0.0, 0,  0, 0], [0.0, 0, 0, 0]],
                        [[1.0,  0, 0, 0], [1.0, 0, -2, 0], [1.0, 0, 0, 0]]])

In [ ]:
Multi_Species_SF_etching.react_table = react_table

In [ ]:
Multi_Species_SF_etching.react_table

In [ ]:
etchfilm[20, 20, :, 0]

In [ ]:
etchfilm[20, 20, :, 0]

In [ ]:
T = 300
Cm = (2*1.380649e-23*T/(27*1.66e-27) )**0.5 # (2kT/m)**0.5 27 for the Al

def max_velocity_u( random1, random2):
    return Cm*np.sqrt(-np.log(random1))*(np.cos(2*np.pi*random2))

def max_velocity_w( random1, random2):
    return Cm*np.sqrt(-np.log(random1))*(np.sin(2*np.pi*random2))

def max_velocity_v( random3):
    return -Cm*np.sqrt(-np.log(random3))

In [ ]:
def velGenerator_benchmark_normal(IN):
    velosity_matrix = np.zeros((IN, 3))
    velosity_matrix[:, 0] = np.random.randn(IN)*0.01 - 0.005
    velosity_matrix[:, 1] = -np.sqrt(2)/2
    velosity_matrix[:, 2] = -np.sqrt(2)/2
    energy = np.linalg.norm(velosity_matrix, axis=1)
    velosity_matrix[:,0] = np.divide(velosity_matrix[:,0], energy)
    velosity_matrix[:,1] = np.divide(velosity_matrix[:,1], energy)
    velosity_matrix[:,2] = np.divide(velosity_matrix[:,2], energy)
    return velosity_matrix

In [ ]:
# F+ benchmark

N = int(1e7)

velosity_matrix = velGenerator_benchmark_normal(N)

typeID = np.zeros(N)

vel_type_shuffle = np.zeros((N, 4))
vel_type_shuffle[:, :3] = velosity_matrix
vel_type_shuffle[:, -1] = typeID

np.random.shuffle(vel_type_shuffle)
# print(typeID[:10])
# print(velosity_matrix[0])
print(vel_type_shuffle[:10])

In [ ]:
# O2

N = int(1e7)
velosity_matrix = np.zeros((N, 3))
# tstep=1e-5
# celllength=1e-5
# velosity_matrix[:, 0] = -1 * celllength /tstep
# velosity_matrix[:, 1] = -1 * celllength /tstep
Random1 = np.random.rand(N)
Random2 = np.random.rand(N)
Random3 = np.random.rand(N)
velosity_matrix = np.array([max_velocity_u(Random1, Random2), \
                            max_velocity_w(Random1, Random2), \
                                max_velocity_v(Random3)]).T

energy = np.linalg.norm(velosity_matrix, axis=1)
velosity_matrix[:,0] = np.divide(velosity_matrix[:,0], energy)
velosity_matrix[:,1] = np.divide(velosity_matrix[:,1], energy)
velosity_matrix[:,2] = np.divide(velosity_matrix[:,2], energy)

# velosity_matrix[:, 2] = -1 * celllength /tstep

typeID = np.zeros(N)
# FO_ratio = int(N/4)
# typeID[-FO_ratio:] = 1

# ion_ration = int(N/8)
# typeID[-ion_ration:] = 2
# velosity_matrix[-ion_ration:, 0] = np.random.rand(ion_ration)*0.001
# velosity_matrix[-ion_ration:, 1] = np.random.rand(ion_ration)*0.001
# velosity_matrix[-ion_ration:, 2] = -1 

vel_type_shuffle = np.zeros((N, 4))
vel_type_shuffle[:, :3] = velosity_matrix
vel_type_shuffle[:, -1] = typeID

np.random.shuffle(vel_type_shuffle)
# print(typeID[:10])
# print(velosity_matrix[0])
print(vel_type_shuffle[:10])

In [ ]:
# F+ Ar+ 
N = int(1e7)
velosity_matrix = np.zeros((N, 3))

typeID = np.zeros(N)

ion_ration = int(N/8)
typeID[-ion_ration:] = 2
velosity_matrix[:, 0] = np.random.rand(N)*0.001
velosity_matrix[:, 1] = np.random.rand(N)*0.001
velosity_matrix[:, 2] = -1 

vel_type_shuffle = np.zeros((N, 4))
vel_type_shuffle[:, :3] = velosity_matrix
vel_type_shuffle[:, -1] = typeID

np.random.shuffle(vel_type_shuffle)
# print(typeID[:10])
# print(velosity_matrix[0])
print(vel_type_shuffle[:10])

In [ ]:
etching = testEtch.inputParticle(125,vel_type_shuffle,int(1e4),int(1e5),int(4e6),-1, 5)

In [ ]:
etching = testEtch.inputParticle(125,vel_type_shuffle,int(1e3),int(4e6),int(1e6),-1, 5)

In [ ]:
# step1 = testEtch.inputParticle(etchfilm, parcel, 'maxwell', 0, int(4e4), int(4e6), int(12e5),2, 10, 100)
etching = testEtch.inputParticle(125,vel_type_shuffle,int(1e4),int(5e5),int(1e6),-1, 5)

In [ ]:
etchfilm[20, 10, :, 0]

In [ ]:
depo1 = torch.Tensor(np.logical_and(etchfilm[:, :, :, 0]==0, film[:, :, :, 0]==10)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(np.logical_and(etchfilm[:, :, :, 0]>0, film[:, :, :, 0]==10)).to_sparse()
substrute = substrute.indices().numpy().T

depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(np.logical_and(etchfilm[:, :, :, 0]==0, film[:, :, :, 0]==10)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(etchfilm[:, :, :, 0]>0).to_sparse()
substrute = substrute.indices().numpy().T

depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
# p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

In [ ]:
depo1 = torch.Tensor(np.logical_and(etchfilm[:, :, :, 0]==10, film[:, :, :, 0]==0)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(film[:, :, :, 0]==10).to_sparse()
substrute = substrute.indices().numpy().T

depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()